In [ ]:
import tensorflow as tf 

import matplotlib.pyplot as plt
import numpy as np

from sionna.phy.ofdm import ResourceGrid
from sionna.phy.channel.tr38901 import CDL, PanelArray
from sionna.phy.channel import OFDMChannel
from sionna.phy.channel.utils import subcarrier_frequencies, cir_to_ofdm_channel

In [ ]:
import sionna
sionna.__version__
tf.__version__

In [ ]:
# Panel array configuration for the transmitter and receiver

rg = ResourceGrid(
    num_ofdm_symbols=32,
    fft_size=64,
    subcarrier_spacing=30e3,
    num_tx=1,
    num_streams_per_tx=1,
    cyclic_prefix_length=1
)

BS_ARRAY = PanelArray(num_rows_per_panel = 64,
    num_cols_per_panel = 1,
    polarization = 'single',
    polarization_type = 'V',
    antenna_pattern = 'omni',
    carrier_frequency = 3.5e9)

UT_ARRAY = PanelArray(num_rows_per_panel = 1,
    num_cols_per_panel = 1,
    polarization = 'single',
    polarization_type = 'V',
    antenna_pattern = 'omni',
    carrier_frequency = 3.5e9)


In [ ]:
1/ rg.ofdm_symbol_duration, rg._subcarrier_spacing, rg.ofdm_symbol_duration

In [ ]:
# Channel Model
cdl = CDL(model="D",
        delay_spread=300e-9,
        carrier_frequency=3.5e9,
        ut_array=UT_ARRAY,
        bs_array=BS_ARRAY,
        min_speed=100,
        max_speed=None,
        direction="downlink")

# Note:  min_speed : float, (default 0.0)
#     Minimum speed [m/s]
# max_speed : `None` (default) | `float`
#     Maximum speed [m/s]. If set to `None`,
#     then ``max_speed`` takes the same value as ``min_speed``.

In [ ]:
# h, tau = cdl(batch_size=5,
#     num_time_steps=10,
#     sampling_frequency=30e6)

# h.shape, tau.shape

# frequency = subcarrier_frequencies(rg.fft_size,
#                                 rg._subcarrier_spacing,
#                                 None)

# frequency

# h_freq = cir_to_ofdm_channel(frequencies=frequency, a=h, tau=tau, normalize=False)

In [ ]:
channel = OFDMChannel(
    channel_model=cdl,
    resource_grid=rg,
    return_channel=True
)

In [ ]:
y, h_freq = channel(tf.zeros([10, 1, 64, rg.num_ofdm_symbols, rg.fft_size], dtype=tf.complex64))
y.shape, h_freq.shape

# Output: y-> Channel output, h_freq -> Channel freq response
# Output
#     -------
#     y : [batch size, num_rx, num_rx_ant, num_ofdm_symbols, fft_size], `tf.complex`
#         Channel outputs
#     h_freq : [batch size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_ofdm_symbols, fft_size], `tf.complex`
#         (Optional) Channel frequency responses

### Converting it to Delay - Angle

In [ ]:
# Step 1: IFFT over delay (last dimension)
h_dd = tf.signal.ifft(h_freq)  # shape: [B, Rx, RxAnt, Tx, TxAnt, Doppler, Delay]
h_dd = tf.signal.ifftshift(h_dd, axes=-1)
# Step 2: FFT over Doppler (2nd-last dimension)
h_dd = tf.transpose(h_dd, perm=[0, 1, 2, 3, 6, 5, 4])  # move Antennas to last
h_dd = tf.signal.fft(h_dd) # FFT over Antenna axis
h_dd = tf.signal.fftshift(h_dd, axes=-1)
h_dd = tf.transpose(h_dd, perm=[0, 1, 2, 3, 4, 5, 6])  # restore original order

In [ ]:
h_tf_reduced = tf.squeeze(h_freq)
print(h_tf_reduced.shape)
h_tf_reordered = tf.transpose(h_tf_reduced, perm=[0, 2, 1, 3])
print(h_tf_reordered.shape)

In [ ]:
h_dd_reduced = tf.squeeze(h_dd) # Removes the dimesnion of size 1
print(h_dd_reduced.shape)
h_dd_reordered = tf.transpose(h_dd_reduced, perm=[0, 2, 3, 1])
print(h_dd_reordered.shape)

In [ ]:
# Plotting/ verifying data from loaded tensors.
fig = plt.figure(figsize=(18, 25))  # Wider and taller figure for 5x2 layout

for i in range(2):
    # --- Time-Frequency (Left Column) ---
    h_tf_mag = tf.abs(h_tf_reordered[1, i*255, :, :]).numpy()
    time_bins, freq_bins = h_tf_mag.shape
    X_tf, Y_tf = np.meshgrid(np.arange(freq_bins), np.arange(time_bins))
                                    
    ax1 = fig.add_subplot(5, 2, 2*i + 1, projection='3d')
    ax1.plot_surface(X_tf, Y_tf, h_tf_mag, cmap='viridis')
    ax1.set_title(f"TimeStep {i*255} - Frequency Space")     
    ax1.set_xlabel("SubCarriers")
    ax1.set_ylabel("Tx Antennas")
    ax1.set_zlabel("|h_tf|")

    # --- Delay-Doppler (Right Column) ---
    h_dd_mag = tf.abs(h_dd_reordered[1, i*255, :, :]).numpy()
    doppler_bins, delay_bins = h_dd_mag.shape
    X_dd, Y_dd = np.meshgrid(np.arange(delay_bins), np.arange(doppler_bins))

    
    ax2 = fig.add_subplot(5, 2, 2*i + 2, projection='3d')
    ax2.plot_surface(X_dd, Y_dd, h_dd_mag, cmap='viridis')
    ax2.set_title(f"TimeStep {i*255} - Delay Angle")
    ax2.set_xlabel("Delay")
    ax2.set_ylabel("Angle")                                    
    ax2.set_zlabel("|h_dd|")

plt.tight_layout()
# plt.savefig('CSI - CDL.png')
plt.show()

### Differences plot

In [ ]:
# Plotting/ verifying data from loaded tensors.
fig = plt.figure(figsize=(18, 25))  # Wider and taller figure for 5x2 layout

for i in range(2):
    # --- Time-Frequency (Left Column) ---
    h_tf_mag = tf.abs(h_tf_reordered[1, i*255, :, :]).numpy()
    time_bins, freq_bins = h_tf_mag.shape
    X_tf, Y_tf = np.meshgrid(np.arange(freq_bins), np.arange(time_bins))
    
    if i == 1:
        h_tf_mag = tf.abs(h_tf_reordered[1, 2, :, :]).numpy() - tf.abs(h_tf_reordered[1, 0, :, :]).numpy()
                                    
    ax1 = fig.add_subplot(5, 2, 2*i + 1, projection='3d')
    ax1.plot_surface(X_tf, Y_tf, h_tf_mag, cmap='viridis')
    ax1.set_title(f"TimeStep {i*255} - Frequency Space")     
    ax1.set_xlabel("SubCarriers")
    ax1.set_ylabel("Tx Antennas")
    ax1.set_zlabel("|h_tf|")

    # --- Delay-Doppler (Right Column) ---
    h_dd_mag = tf.abs(h_dd_reordered[1, i*255, :, :]).numpy()
    doppler_bins, delay_bins = h_dd_mag.shape
    X_dd, Y_dd = np.meshgrid(np.arange(delay_bins), np.arange(doppler_bins))

    if i == 1:
        h_dd_mag = tf.abs(h_dd_reordered[1, 2, :, :]).numpy() - tf.abs(h_dd_reordered[1, 0, :, :]).numpy()
    
    ax2 = fig.add_subplot(5, 2, 2*i + 2, projection='3d')
    ax2.plot_surface(X_dd, Y_dd, h_dd_mag, cmap='viridis')
    ax2.set_title(f"TimeStep {i*255} - Delay Angle")
    ax2.set_xlabel("Delay")
    ax2.set_ylabel("Angle")                                    
    ax2.set_zlabel("|h_dd|")

plt.tight_layout()
# plt.savefig('CSI - CDL.png')
plt.show()

### Converting it to Delay Doppler

In [ ]:
# Step 1: IFFT over delay (last dimension)
h_dd = tf.signal.ifft(h_freq)  # shape: [B, Rx, RxAnt, Tx, TxAnt, Doppler, Delay]
h_dd = tf.signal.ifftshift(h_dd, axes=-1)
# Step 2: FFT over Doppler (2nd-last dimension)
h_dd = tf.transpose(h_dd, perm=[0, 1, 2, 3, 4, 6, 5])  # move Doppler to last
h_dd = tf.signal.fft(h_dd) # FFT over Doppler axis
h_dd = tf.signal.fftshift(h_dd, axes=-1)
h_dd = tf.transpose(h_dd, perm=[0, 1, 2, 3, 4, 5, 6])  # restore original order

In [ ]:
h_tf_reduced = tf.squeeze(h_freq)
print(h_tf_reduced.shape)

h_dd_reduced = tf.squeeze(h_dd) # Removes the dimesnion of size 1
print(h_dd_reduced.shape)

h_dd_reordered = tf.transpose(h_dd_reduced, perm=[0, 1, 3, 2])
print(h_dd_reordered.shape)

In [ ]:
N_sym = rg.num_ofdm_symbols
T_sym = rg.ofdm_symbol_duration

f_d = 1 / (N_sym * T_sym)

v_d = 0.08571 * f_d

In [ ]:
# # Plotting/ verifying data from loaded tensors.
# fig = plt.figure(figsize=(18, 25))  # Wider and taller figure for 5x2 layout

# for i in range(2):
#     # --- Time-Frequency (Left Column) ---
#     h_tf_mag = tf.abs(h_tf_reduced[i, 0, :, :]).numpy()
#     time_bins, freq_bins = h_tf_mag.shape
#     X_tf, Y_tf = np.meshgrid(np.arange(freq_bins), np.arange(time_bins))

#     ax1 = fig.add_subplot(5, 2, 2*i + 1, projection='3d')
#     ax1.plot_surface(X_tf, Y_tf, h_tf_mag, cmap='viridis')
#     ax1.set_title(f"Sample {i+1} - Frequency Time")
#     ax1.set_xlabel("Frequency")
#     ax1.set_ylabel("Time")
#     ax1.set_zlabel("|h_tf|")

#     # --- Delay-Doppler (Right Column) ---
#     h_dd_mag = tf.abs(h_dd_reordered[i, 0, :, :]).numpy()
#     doppler_bins, delay_bins = h_dd_mag.shape
#     X_dd, Y_dd = np.meshgrid(np.arange(delay_bins), np.arange(doppler_bins))
#     Y_dd = (Y_dd - len(Y_dd) /2)*f_d
    
#     ax2 = fig.add_subplot(5, 2, 2*i + 2, projection='3d')
#     ax2.plot_surface(X_dd, Y_dd, h_dd_mag, cmap='viridis')
#     ax2.set_title(f"Sample {i+1} - Delay Doppler")
#     ax2.set_xlabel("Delay")
#     ax2.set_ylabel("Doppler")                                    
#     ax2.set_zlabel("|h_dd|")

# plt.tight_layout()
# # plt.savefig('CSI - CDL.png')
# plt.show()

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(18, 25))  # Wider and taller figure for 5x2 layout

# Optional: enable interactive 3D rotation if you're in Jupyter
# %matplotlib widget

for i in range(5):  # change to range(5) if you want 5 samples
    # --- Time-Frequency (Left Column) ---
    h_tf_mag = tf.abs(h_tf_reduced[i, 0, :, :]).numpy()
    time_bins, freq_bins = h_tf_mag.shape
    X_tf, Y_tf = np.meshgrid(np.arange(freq_bins), np.arange(time_bins))

    ax1 = fig.add_subplot(5, 2, 2*i + 1, projection='3d')
    surf1 = ax1.plot_surface(X_tf, Y_tf, h_tf_mag, cmap='viridis')

    # Find and mark peak
    peak_idx = np.unravel_index(np.argmax(h_tf_mag), h_tf_mag.shape)
    x_peak, y_peak, z_peak = X_tf[peak_idx], Y_tf[peak_idx], h_tf_mag[peak_idx]

    ax1.scatter(x_peak, y_peak, z_peak, color='r', s=50, label=f"Peak ({x_peak}, {y_peak}, {z_peak:.2f})")
    ax1.legend(loc='upper right')

    ax1.set_title(f"Sample {i+1} - Frequency Time")
    ax1.set_xlabel("Frequency")
    ax1.set_ylabel("Time")
    ax1.set_zlabel("|h_tf|")

    # --- Delay-Doppler (Right Column) ---
    h_dd_mag = tf.abs(h_dd_reordered[i, 0, :, :]).numpy()
    doppler_bins, delay_bins = h_dd_mag.shape
    X_dd, Y_dd = np.meshgrid(np.arange(delay_bins), np.arange(doppler_bins))
    Y_dd = (Y_dd - len(Y_dd) / 2) * f_d

    ax2 = fig.add_subplot(5, 2, 2*i + 2, projection='3d')
    surf2 = ax2.plot_surface(X_dd, Y_dd, h_dd_mag, cmap='viridis')

    # Find and mark peak
    peak_idx_dd = np.unravel_index(np.argmax(h_dd_mag), h_dd_mag.shape)
    x_peak_dd, y_peak_dd, z_peak_dd = X_dd[peak_idx_dd], Y_dd[peak_idx_dd], h_dd_mag[peak_idx_dd]

    ax2.scatter(x_peak_dd, y_peak_dd, z_peak_dd, color='r', s=50, label=f"Peak ({x_peak_dd}, {y_peak_dd:.2f}, {z_peak_dd:.2f})")
    ax2.legend(loc='upper right')

    ax2.set_title(f"Sample {i+1} - Delay Doppler")
    ax2.set_xlabel("Delay")
    ax2.set_ylabel("Doppler (Hz)")
    ax2.set_zlabel("|h_dd|")

plt.tight_layout()
plt.show()
